# ALGORITMO DE OPTIMIZACIÓN

# Importación de librerías y definición de funciones

Lo primero que haremos, es importar todas las librerías que se utilizaran a lo largo del Notebook y definir todas las funciones que usaremos.

In [ ]:
# Importamos todas las librerías que se utilizarán en el Notebook

# Librerías para manejo de datos
import pandas as pd
import numpy as np

# Librerías para visualización gráfica
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de SciKit Learn para realizar Machine Learning
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score

# Librerías para Boosting con datos desbalanceados
try: 
    from xgboost import XGBClassifier
except ImportError:
    print("Para correr este Notebook precisas instalar la librería XGBoost. Para esto, ingresa el siguiente comando en la consola de comandos: pip install xgboost")
except ModuleNotFoundError:
    print("Para correr este Notebook precisas instalar la librería XGBoost. Para esto, ingresa el siguiente comando en la consola de comandos: pip install xgboost")
    
# Librería para manejo del tiempo
import time


"""
Función mostrar_resultados()

Se utiliza para visualizar los resultados de una clasificación,

** Argumentos
    y_test: las etiquetas de prueba
    y_pred: las etiquetas predichas mediante el modelo de clasificación

** Devuelve
    Impresión de la matriz de confusión y del reporte de clasificación
"""
def mostrar_resultados(y_test, y_pred):
    plt.figure(figsize=(6, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred), xticklabels=['NO', 'SI'], yticklabels=['NO', 'SI'], annot=True, fmt="d")
    plt.title("Matriz de Confusión")
    plt.ylabel('Clases reales')
    plt.xlabel('Clases predichas')
    plt.show()
    print(classification_report(y_test, y_pred))





"""
Función mostrar_analisis_PCA()

Se utiliza para descomponer una matriz en sus componentes principales

** Argumentos
    X: matriz a analizar/descomponer

** Devuelve
    Componentes principales de X
"""
def analisis_PCA(X):
    model = PCA(n_components = 3)
    model.fit(X)
    return model.transform(X)
    

    
"""
Función buscar_mejores_parametros()

Se utiliza para buscar, aleatoriamente mediante RandomizedSearchCV(),
la configuración de parámetros del clasificador que maximizan 
la precisión balanceada (balanced_accuracy).

Esto se logra mediante la iteración en un rango específico de parámetros,
que se definió aleatoriamente para sacarle mayor provecho a RandomizedSearchCV().

** Argumentos
    X_train: la matriz con los datos de entrenamiento
    y_train: las etiquetas de los datos de entrenamiento
    metodo: el método que sobre el que se iterarán los parámetros
            RFC = RandomForestClassifier() -> método por default
            DTC = DecisionTreeClassifier()
            LR  = LogisticRegression()
            GBC = GradientBoostingClassifier()
            ETC = ExtraTreesClassifier()
            ABC = AdaBoostClassifier()
            XGB = XGBClassifier()
    iters: número de iteraciones a realizar. Por default 4.

** Devuelve
    1. El estimador ya configurado con los parámetros que maximizan la métrica balanced_accuracy.
    2. Los mejores parámetros encontrados.
    3. El resultado de balanced_accuracy (maximizado).
    4. Resultados de la validación cruzada
"""
def buscar_mejores_parametros(X_train, y_train, metodo = 'RFC', iters = 4):
    
    le = LabelEncoder()
    le.fit(y_train.unique())
    y_train_aux = le.transform(y_train)
    
    if metodo == 'RFC':
        estimador = RandomForestClassifier(n_jobs = -1, random_state = 7953)
    elif metodo == 'DTC':
        estimador = DecisionTreeClassifier(random_state = 7953)
    elif metodo == 'LR':
        estimador = LogisticRegression(n_jobs = -1, random_state = 7953)
    elif metodo == 'GBC':
        estimador = GradientBoostingClassifier(random_state = 7953)
    elif metodo == 'ETC':
        estimador = ExtraTreesClassifier(n_jobs = -1, random_state = 7953)
    elif metodo == 'ABC':
        estimador = AdaBoostClassifier(random_state = 7953)
    elif metodo == 'XGB':
        relacion  = max(y_train.value_counts()) / min(y_train.value_counts())
        estimador = XGBClassifier(n_jobs = -1, random_state = 7953, scale_pos_weight = relacion)

    params = {
                 'RFC' : {
                             'n_estimators'     : np.random.randint(0, 100, size = 20),
                             'criterion'        : ('gini', 'entropy'),
                             'max_depth'        : np.random.randint(0,  75, size = 20),
                             'min_samples_split': np.random.randint(0, 250, size = 20),
                             'min_samples_leaf' : np.random.randint(0, 250, size = 20),
                             'class_weight'     : ('balanced', 'balanced_subsample', None),
                             'max_features'     : ('auto', 'sqrt', 'log2', None)
                         },
        
                 'DTC' : {
                             'criterion'        : ('gini', 'entropy'),
                             'splitter'         : ('best', 'random'),
                             'max_depth'        : np.random.randint(0, 100, size = 25),
                             'min_samples_split': np.random.randint(0, 300, size = 25),
                             'min_samples_leaf' : np.random.randint(0, 300, size = 25),
                             'class_weight'     : ('balanced', None),
                             'max_features'     : ('auto', 'sqrt', 'log2', None)
                         },
        
                 'LR' : {
                             'C'                : np.random.uniform(0, 1, size = 15),
                             'fit_intercept'    : (True, False),
                             'class_weight'     : ('balanced', None),
                             'solver'           : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'),
                             'max_iter'         : np.random.randint(0, 1000, size = 25),
                             'l1_ratio'         : np.random.uniform(0, 1   , size = 15)
                        },
                
                'GBC' : {
                             'loss'             : ('deviance', 'exponential'),
                             'learning_rate'    : np.random.uniform(0, 1  , size = 15),
                             'n_estimators'     : np.random.randint(0, 150, size = 15),
                             'subsample'        : np.random.uniform(0, 1  , size =  5),
                             'criterion'        : ('friedman_mse', 'mse', 'mae'),
                             'min_samples_split': np.random.randint(0, 150, size = 15),
                             'min_samples_leaf' : np.random.randint(0, 150, size = 15),
                             'max_features'     : ('auto', 'sqrt', 'log2', None),
                             'warm_start'       : (True, False)
                        },
                
                'ETC' : {
                             'n_estimators'     : np.random.randint(0, 300, size = 25),
                             'criterion'        : ('gini', 'entropy'),
                             'max_depth'        : np.random.randint(0, 100, size = 25),
                             'min_samples_split': np.random.randint(0, 300, size = 25),
                             'min_samples_leaf' : np.random.randint(0, 300, size = 25),
                             'bootstrap'        : (True, False),
                             'class_weight'     : ('balanced', 'balanced_subsample', None)
                        }, 
        
                'ABC' : {
                             'n_estimators'     : np.random.randint(1, 50, size = 15),
                             'learning_rate'    : np.random.uniform(0, 1 , size = 15) 
                        },
            
                'XGB' : {
                             'n_estimators'     : np.random.randint(1, 50, size = 20),
                             'max_depth'        : np.random.randint(1, 40, size = 15),
                             'learning_rate'    : np.random.uniform(0, 2 , size = 20),
                             'objective'        : ('binary:logistic', 'binary:logitraw', 'reg:logistic'),
                             'reg_lambda'       : np.random.uniform(0, 3 , size = 5),
                             'reg_alpha'        : np.random.uniform(0, 3 , size = 5),
                             'gamma'            : np.random.uniform(0, 10, size = 5),  
                             'min_child_weight' : np.random.randint(1, 10, size = 10), 
                             'max_delta_step'   : np.random.randint(1, 10, size = 10)
                        }
             }

    scorers = {
                  'accuracy'         : 'accuracy',
                  'balanced_accuracy': 'balanced_accuracy',
                  'f1'               : 'f1',
                  'neg_log_loss'     : 'neg_log_loss',
                  'precision'        : 'precision',
                  'recall'           : 'recall'
              }

    rs = RandomizedSearchCV(
                                estimator           = estimador,
                                param_distributions = params[metodo], 
                                scoring             = scorers, 
                                refit               = 'balanced_accuracy', 
                                n_iter              = iters, 
                                n_jobs              = -1,
                                random_state        = 7953,
                                cv                  = 10
                           )

    rs.fit(X_train, y_train_aux)
    
    return rs.best_estimator_, rs.best_params_, rs.best_score_, rs.cv_results_

# 1. Carga del Dataset y exploración de datos
Ahora, cargamos el dataset dentro de un DataFrame de Pandas, y visualizamos algunos valores

In [ ]:
# Creamos el DataFrame a partir del DataSet enviado
mkt = pd.read_csv('mkt_bank.csv')

# Inspeccionamos la estructura de los datos
mkt.head()

Miramos qué clase de datos contiene cada columna y si están completas o no

In [ ]:
# Nos fijamos si todas las columnas están completas
# y qué tipos de datos contienen
mkt.info()

In [ ]:
# Realizamos un análisis estadístico
mkt.describe()

Para tener una idea de la cantidad de datos categóricos desconocidos que hay en el DataFrame, imprimimos, columna por columna, cuántas veces aparece el string "desconocido".

In [ ]:
# Primero filtramos todas las columnas que tienen valores categóricos
cols_categ = mkt.dtypes == object

# Luego, obtenemos los nombres de estas columnas
cols_categ_index = cols_categ.loc[cols_categ.values == True].index

for categ in cols_categ_index:
    print('Cantidad de desconocidos en la columna "' + str(categ) + '": ' + str(mkt[mkt[categ] == 'desconocido'].shape[0]))

# 2. Verificación de la distribución de clases

Lo siguiente se hace para tener una idea de la distribución de clases que tiene el dataset. Como se aprecia, sólo el 11% son positivos, por lo que esto nos da una idea del modelo nulo y del alto desbalanceo de la información.

In [ ]:
print('Cantidades por clase en el dataset: ')
print(mkt['y'].value_counts())

print('\n')

print('Proporción de clases en el dataset: ')
print(mkt['y'].value_counts(normalize = True))

# 3. Pre-procesamiento de datos

Realizamos a continuación un pre-procesamiento de los datos, con el fin de mejorar la performance del método de clasificación que usaremos luego.

In [ ]:
# Eliminamos la primer columna del DataFrame, que no es útil para el análisis
mkt = mkt.drop(columns = ['Unnamed: 0'], axis = 1)

# Asignamos la variable objetivo
y = mkt['y']

# Luego la eliminamos del DataFrame
mkt = mkt.drop(columns = ['y'], axis = 1)

# Filtramos todas las columnas que tienen valores categóricos
cols_categ = mkt.dtypes == object

# Luego, obtenemos los nombres de estas columnas
cols_categ_index = cols_categ.loc[cols_categ.values == True].index

# Vamos a utilizar LabelEncoder() para pasar a numéricas todas las variables categóricas

# En el siguiente bucle, transformamos todas las variables con LabelEncoder()
for col in cols_categ_index:
    le = LabelEncoder()
    le.fit(mkt[col].unique())
    mkt[col] = le.transform(mkt[col])

# En la columna 'Dias', 999 significa que el cliente no fue contactado previamente.
# Resulta razonable entonces convertirlo a cero para uniformizar la información
mkt.replace({'Dias': {999: 0}}, inplace = True)

# Vemos cómo quedó el DataFrame
mkt

In [ ]:
# Asignamos a 'X' las variables explicativas
X = mkt

# 4. Estandarización

Hacemos que todas las variables tengan media nula y desvío unitario.

In [ ]:
# Estandarizamos las variables cuantitativas con StandardScaler() (media = 0 y desvío = 1)
cl = ColumnTransformer(transformers = [('Scaler', StandardScaler(), X.columns)],
                       remainder='passthrough')

# Ejecutamos la transformación y visualizamos cómo quedó
X = cl.fit_transform(X)
X

# 5. Separación de los datos en conjuntos de entrenamiento y prueba

Realizamos la separación mediante el siguiente criterio: 80% de los datos para entrenamiento y 20% para prueba.
Dado el alto desbalance de clases presente, utilizamos estratificación en el método, para obtener conjuntos con las mismas proporciones.

In [ ]:
# Separamos en sets de entrenamiento y prueba, usando
# estratificación, dada la gran disparidad de clases.
# Separamos el 20% de los datos para entrenamiento.
X_train, X_test, y_train, y_test = train_test_split(
                                                       X, y,
                                                       test_size    = 0.2,
                                                       stratify     = y,
                                                       random_state = 7953
                                                   )

# 6. Búsqueda de parámetros óptimos

Usaremos la función buscar_mejores_parametros() para optimizar los clasificadores.

Para esto, debemos seleccionar el clasificador que usaremos y la cantidad de iteraciones a realizar.

##### ADVERTENCIA: Un gran número de iteraciones puede llegar a demorar mucho en finalizar, en función de la potencia de cálculo disponible. Para probar, ver cuánto demora con 1 iteración para tener una idea.

In [ ]:
# Configuramos la clase que usaremos para optimizar sus parámetros.
# XGB = XGBClassifier()
clase = 'XGB'

# Elegimos la cantidad de iteraciones a realizar en la optimización
iteraciones = 10

In [ ]:
# Tomamos el tiempo inicial
tic = time.time()

# Buscamos los mejores parámetros para Boosting
xgb, mejores_params, mejor_scorer, cv_res = buscar_mejores_parametros(
                                                                         X_train,
                                                                         y_train,
                                                                         metodo = clase,
                                                                         iters  = iteraciones
                                                                     )
# Tomamos el tiempo final
toc = time.time()

print('Demora de buscar_mejores_parametros() para XGB: {seg:.2f} segundos ({mins:.2f} minutos)\n'.format(seg = toc-tic, mins = (toc-tic)/60))

# Visualizamos cuáles son los mejores parámetros y el scorer
print('Los mejores parámetros de XGB son: ')
print(mejores_params)
print('\nEl máximo scorer de XGB dió: ' + str(mejor_scorer))

In [ ]:
# Visualizamos los resultados de la validación cruzada
print("Resultados de la validación cruzada con estratificación")
pd.DataFrame(cv_res)

Entrenamos al modelo optimizado, predecimos y visualizamos los resultados

In [ ]:
#Entrenamos al modelo
xgb.fit(X_train, y_train)

# Realizamos las predicciones y visualizamos los resultados
y_pred = xgb.predict(X_test)

print('Los resultados con XGBClassifier() son: \n')

# Vemos la exactitud balanceada, útil para los casos de alto desbalanceo de clases
print('El accuracy balanceado de entrenamiento es de: {}'.format(balanced_accuracy_score(y_train, xgb.predict(X_train))))
print('El accuracy balanceado de prueba es de: {}\n'.format(balanced_accuracy_score(y_test, y_pred)))

# Vemos la exactitud convencional
print('El accuracy de entrenamiento es de: {}'.format(accuracy_score(y_train, xgb.predict(X_train))))
print('El accuracy de prueba es de: {}'.format(accuracy_score(y_test, y_pred)))

mostrar_resultados(y_test, y_pred)

In [ ]:
# Calculamos las probabilidades
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p_tp = tp / (tp + fn) # Probabilidad de positivo real
p_fn = fn / (tp + fn) # Probabilidad de falso negativo
p_tn = tn / (tn + fp) # Probabilidad de negativo real
p_fp = fp / (tn + fp) # Probabilidad de falso positivo

# Luego las imprimimos
print('Probabilidad de positivo real: {proba:.2f} %'.format(proba = p_tp*100))
print('Probabilidad de falso negativo: {proba:.2f} %'.format(proba = p_fn*100))
print('Probabilidad de negativo real: {proba:.2f} %'.format(proba = p_tn*100))
print('Probabilidad de falso positivo: {proba:.2f} %'.format(proba = p_fp*100))

In [ ]:
# Asignamos el costo/beneficio de cada posibilidad
b_tp = 2900
b_tn = 200
c_fp = -440
c_fn = 0

# Visualizamos la probabilidad de cada caso
proporcion = y.value_counts(normalize = True)

# Finalmente, calculamos la esperanza
clase_0 = proporcion[0]
clase_1 = proporcion[1]
valor_esperado = clase_1 * (p_tp * b_tp + p_fn * c_fn) + clase_0 * (p_tn * b_tn + p_fp * c_fp)
valor_esperado